In [72]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jun  4 21:59:36 2023

@author: abdullahalbinsaleh
"""

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 20 06:20:43 2023

@author: abdullahalbinsaleh

This version uses normal random data splitting technique 
"""

#-------------------------import librarires-------------------------#
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import RFECV


In [ ]:

#-------------------------Load dataset-------------------------#
moneyFlow = pd.read_excel("moneyFlow.xlsx")
moneyFlow.drop('Unnamed: 0', axis=1, inplace=True)
#-------------------------Data Exploration-------------------------#
moneyFlow.head()
moneyFlow.info()
moneyFlow.columns


#-------------------------Data Preparation & Pre-Processing-------------------------#
#drop $ sign from Spot feature 
# convert the 'Spot' column to a string
moneyFlow['Spot'] = moneyFlow['Spot'].astype(str)
moneyFlow['Spot'] = moneyFlow['Spot'].str.replace("$", "")

# convert the 'Spot' column back to float
moneyFlow['Spot'] = moneyFlow['Spot'].astype(float)


#Split Details feature into two features 
moneyFlow[["Size", "Price"]] = moneyFlow["Details"].str.split("@", expand=True)

#drop the Details feature 
moneyFlow.drop(columns=["Details"], inplace=True)

#Find options duration in days 
moneyFlow["Date"] = pd.to_datetime(moneyFlow["Date"])
moneyFlow["Expiration"] = pd.to_datetime(moneyFlow["Expiration"])
moneyFlow["Duration"] = (moneyFlow["Expiration"]-moneyFlow["Date"]).dt.days

#drop the Date, Expiration, and Time Featuers 
moneyFlow.drop(columns=["Time", "Date", "Expiration", "Ticker"], inplace=True)


#Convert categoircal variables into dummy variables 
moneyFlow = pd.get_dummies(moneyFlow, columns=["Type", "Execution", "C/P"])

#Drop missing values 
moneyFlow.dropna(inplace=True)


#-------------------------Feature Selection-------------------------#
# Separate features and target variable

moneyFlow = moneyFlow[moneyFlow['Sentiment'] != 'NEUTRAL']


#-------------------------Feature Selection-------------------------#
# Separate features and target variable

# We get the same results as below but with less work and less chance of error

#X_lazy = moneyFlow[list(selected_features)]

#assign selected features using RFE to X
X = moneyFlow[['Strike', 'Spot', 'Open Interest', 'Size', 'Price', 'Duration',
       'Type_SWEEP', 'Execution_ABOVE ASK', 'Execution_AT ASK',
       'Execution_AT BID', 'Execution_AT MIDPOINT', 'Execution_BELOW BID',
       'C/P_CALL', 'C/P_PUT']]
#X = moneyFlow[['Execution_AT BID', 'Execution_BELOW BID', 'C/P_PUT']]
#-------------------------Data Split-------------------------#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Initialize the classifier


# You were using fit_transform for both training and testing data, this is incorrect 
# we should only fir the scaler to the training data and then transform both training and testing data using the fit

#-------------------------Feature Scaling-------------------------#
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



#-------------------------Logisitc Regression-------------------------#
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

#-------------------------Logisitc Regression Classification Report and Confusion Matrix-------------------------#
print("Logistic Regression classification report:")
print(classification_report(y_test, lr_pred))
print("Logistic Regression confusion matrix:")
print(confusion_matrix(y_test, lr_pred))



In [68]:

# Random Forest may be too Complex and have a tendency to overfit especially if they dont have their hyperparemters optimally tuned
# Using Decision Tree Classification
#-------------------------Decision Tree Classification-------------------------#
rf = DecisionTreeClassifier(max_depth=2, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

#-------------------------A Decision Tree Classification Report and Confusion Matrix-------------------------#
print("A Decision Tree Classification Report:")
print(classification_report(y_test, rf_pred))
print("A Decision Tree confusion matrix:")
print(confusion_matrix(y_test, rf_pred))

#-------------------------Gaussian Naive Bayes Classification-------------------------#

nn = GaussianNB()

nn.fit(X_train, y_train)
nn_pred = nn.predict(X_test)


#-------------------------Gaussian Naive Bayes Classification Report and Confusion Matrix-------------------------#
print("Gaussian Naive Bayes report:")
print(classification_report(y_test, nn_pred))
print("Gaussian Naive Bayes confusion matrix:")
print(confusion_matrix(y_test, nn_pred))



#-------------------------SGD-------------------------#
SGDC = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
SGDC.fit(X_train, y_train)
SGDC_pred = SGDC.predict(X_test)
#-------------------------SGD Classification Report and Confusion Matrix-------------------------#
print("SGDC Confusion Matrix:")
print(confusion_matrix(y_test, SGDC_pred))
print("\nStochastic Gradient Descent Report:")
print(classification_report(y_test, SGDC_pred))



A Decision Tree Classification Report:
              precision    recall  f1-score   support

     BEARISH       1.00      1.00      1.00     25114
     BULLISH       1.00      1.00      1.00     24936

    accuracy                           1.00     50050
   macro avg       1.00      1.00      1.00     50050
weighted avg       1.00      1.00      1.00     50050

A Decision Tree confusion matrix:
[[25114     0]
 [    0 24936]]
Gaussian Naive Bayes report:
              precision    recall  f1-score   support

     BEARISH       0.60      0.60      0.60     25114
     BULLISH       0.60      0.59      0.59     24936

    accuracy                           0.60     50050
   macro avg       0.60      0.60      0.60     50050
weighted avg       0.60      0.60      0.60     50050

Gaussian Naive Bayes confusion matrix:
[[15173  9941]
 [10317 14619]]
SGDC Confusion Matrix:


c:\Users\JBarr\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[[15173  9941]
 [    0 24936]]

Stochastic Gradient Descent Report:
              precision    recall  f1-score   support

     BEARISH       1.00      0.60      0.75     25114
     BULLISH       0.71      1.00      0.83     24936

    accuracy                           0.80     50050
   macro avg       0.86      0.80      0.79     50050
weighted avg       0.86      0.80      0.79     50050



In [80]:
X = moneyFlow.drop("Sentiment", axis=1)
y = moneyFlow["Sentiment"]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Overfitting tip 

# There is no need for data scaling if we are using decision trees and random forests as they are not sensitive to scale of input
#  this method might be selecting too many features, leading to a complex model that could overfit.
classifier = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=42)

# Initialize RFECV with the classifier
rfecv = RFECV(estimator=classifier, step=1, cv=5)

# Perform feature selection
X_selected = rfecv.fit_transform(X_scaled, y)


# Print the selected features
selected_features = X.columns[rfecv.support_]
print("Selected Features:")
print(selected_features)


Selected Features:
Index(['Prem', 'Volume', 'Price', 'Duration', 'Type_TRADE',
       'Execution_ABOVE ASK', 'Execution_AT ASK', 'Execution_AT BID',
       'Execution_BELOW BID', 'C/P_CALL', 'C/P_PUT'],
      dtype='object')


In [86]:

#assign selected features using RFE to X
# X = moneyFlow[['Strike', 'Spot', 'Open Interest', 'Size', 'Price', 'Duration',
#        'Type_SWEEP', 'Execution_ABOVE ASK', 'Execution_AT ASK',
#        'Execution_AT BID', 'Execution_AT MIDPOINT', 'Execution_BELOW BID',
#        'C/P_CALL', 'C/P_PUT']]
X = moneyFlow[list(selected_features)]

#-------------------------Data Split-------------------------#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Initialize the classifier


# You were using fit_transform for both training and testing data, this is incorrect 
# we should only fir the scaler to the training data and then transform both training and testing data using the fit

#-------------------------Feature Scaling-------------------------#
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)




In [87]:

#-------------------------Logisitc Regression-------------------------#
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

#-------------------------Logisitc Regression Classification Report and Confusion Matrix-------------------------#
print("Logistic Regression classification report:")
print(classification_report(y_test, lr_pred))
print("Logistic Regression confusion matrix:")
print(confusion_matrix(y_test, lr_pred))


Logistic Regression classification report:
              precision    recall  f1-score   support

     BEARISH       0.60      0.61      0.60     25114
     BULLISH       0.60      0.59      0.59     24936

    accuracy                           0.60     50050
   macro avg       0.60      0.60      0.60     50050
weighted avg       0.60      0.60      0.60     50050

Logistic Regression confusion matrix:
[[15205  9909]
 [10325 14611]]


In [88]:


# Random Forest may be too Complex and have a tendency to overfit especially if they dont have their hyperparemters optimally tuned
# Using Decision Tree Classification
#-------------------------Decision Tree Classification-------------------------#
rf = DecisionTreeClassifier(max_depth=2, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

#-------------------------A Decision Tree Classification Report and Confusion Matrix-------------------------#
print("A Decision Tree Classification Report:")
print(classification_report(y_test, rf_pred))
print("A Decision Tree confusion matrix:")
print(confusion_matrix(y_test, rf_pred))


A Decision Tree Classification Report:
              precision    recall  f1-score   support

     BEARISH       0.97      0.96      0.97     25114
     BULLISH       0.96      0.97      0.97     24936

    accuracy                           0.97     50050
   macro avg       0.97      0.97      0.97     50050
weighted avg       0.97      0.97      0.97     50050

A Decision Tree confusion matrix:
[[24135   979]
 [  688 24248]]


In [89]:

#-------------------------Gaussian Naive Bayes Classification-------------------------#

nn = GaussianNB()

nn.fit(X_train, y_train)
nn_pred = nn.predict(X_test)


#-------------------------Gaussian Naive Bayes Classification Report and Confusion Matrix-------------------------#
print("Gaussian Naive Bayes report:")
print(classification_report(y_test, nn_pred))
print("Gaussian Naive Bayes confusion matrix:")
print(confusion_matrix(y_test, nn_pred))


Gaussian Naive Bayes report:
              precision    recall  f1-score   support

     BEARISH       0.60      0.60      0.60     25114
     BULLISH       0.59      0.59      0.59     24936

    accuracy                           0.59     50050
   macro avg       0.59      0.59      0.59     50050
weighted avg       0.59      0.59      0.59     50050

Gaussian Naive Bayes confusion matrix:
[[15038 10076]
 [10233 14703]]


In [94]:



#-------------------------SGD-------------------------#
SGDC = SGDClassifier(loss="log_loss", penalty="l2", max_iter=50)
SGDC.fit(X_train, y_train)
SGDC_pred = SGDC.predict(X_test)
#-------------------------SGD Classification Report and Confusion Matrix-------------------------#
print("SGDC Confusion Matrix:")
print(confusion_matrix(y_test, SGDC_pred))
print("\nStochastic Gradient Descent Report:")
print(classification_report(y_test, SGDC_pred))



SGDC Confusion Matrix:
[[15214  9900]
 [10324 14612]]

Stochastic Gradient Descent Report:
              precision    recall  f1-score   support

     BEARISH       0.60      0.61      0.60     25114
     BULLISH       0.60      0.59      0.59     24936

    accuracy                           0.60     50050
   macro avg       0.60      0.60      0.60     50050
weighted avg       0.60      0.60      0.60     50050

